In [26]:
# Section must be included at the beginning of each new notebook. Remember to change the app name. 
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [27]:
# Let's read in the data. If you open the dataset, you'll find that each column has a header. We specify that by stating that header=True.
# To make our lives easier, we can also use 'inferSchema' when importing CSVs. This automatically detects data types.
# If you would like to manually change data types, refer to this article: https://medium.com/@mrpowers/adding-structtype-columns-to-spark-dataframes-b44125409803
df = spark.read.csv('Datasets/IVS_Country.csv',header=True,inferSchema=True)

In [28]:
# The show method allows you visualise DataFrames in a tabular format. 
df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Year ending: string (nullable = true)
 |-- Airport of departure: string (nullable = true)
 |-- Purpose of visit: string (nullable = true)
 |-- Country of permanent residence: string (nullable = true)
 |-- Total visitor spend: double (nullable = true)
 |-- Total visitors: integer (nullable = true)



In [29]:
# Print schema allows us to visualise the data structure at a high level. 
df.printSchema()

# We can also use head to print a specific amount of rows, so we can get a better understanding of the data points. 
# Note that we have to specify 'print' depending on the method we're using. Otherwise it may not show up!
print(df.head(1))


root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Year ending: string (nullable = true)
 |-- Airport of departure: string (nullable = true)
 |-- Purpose of visit: string (nullable = true)
 |-- Country of permanent residence: string (nullable = true)
 |-- Total visitor spend: double (nullable = true)
 |-- Total visitors: integer (nullable = true)

[Row(Year=1997, Quarter=4, Year ending='YEDec 1997', Airport of departure='Auckland', Purpose of visit='Business', Country of permanent residence='Africa and Middle East', Total visitor spend=4266524.07, Total visitors=1684)]


In [30]:

# We can use the describe method get some general statistics on our data too. 
df.describe().show()

+-------+-----------------+------------------+-----------+--------------------+--------------------+------------------------------+--------------------+------------------+
|summary|             Year|           Quarter|Year ending|Airport of departure|    Purpose of visit|Country of permanent residence| Total visitor spend|    Total visitors|
+-------+-----------------+------------------+-----------+--------------------+--------------------+------------------------------+--------------------+------------------+
|  count|            17056|             17056|      17056|               17056|               17056|                         17056|               11292|             11292|
|   mean|2007.499941369606|               2.5|       null|                null|                null|                          null|  5.09246119696264E7|15576.741232731138|
| stddev|5.927085540507713|1.1289220372929016|       null|                null|                null|                          null|1.0193863

In [31]:
# Let's select the columns that are numbers, and use the describe method again.
# We see that the number of visitors is 11292
df.select('total visitor spend', 'total visitors').describe().show()

+-------+--------------------+------------------+
|summary| total visitor spend|    total visitors|
+-------+--------------------+------------------+
|  count|               11292|             11292|
|   mean|  5.09246119696264E7|15576.741232731138|
| stddev|1.0193863986366412E8|33229.303734004156|
|    min|                 2.4|                 1|
|    max|       1.235333092E9|            314584|
+-------+--------------------+------------------+



In [32]:
df.filter("year > 1997 AND year < 2018").select('year','total visitor spend','total visitors').show()

+----+-------------------+--------------+
|year|total visitor spend|total visitors|
+----+-------------------+--------------+
|1998|         3869387.92|          1470|
|1998|      1.058719738E8|         81260|
|1998|         6954459.85|          2528|
|1998|         9494071.42|          4292|
|1998|         8224341.72|          2614|
|1998|       2.89316078E7|         11486|
|1998|         2954720.33|          1481|
|1998|      1.330060993E7|          4016|
|1998|      4.071200398E7|         14803|
|1998|         9591296.92|          5667|
|1998|      2.214571426E7|         10492|
|1998|      2.279319377E7|          7531|
|1998|      7.490013731E7|         17996|
|1998|      5.459006734E7|         18851|
|1998|      1.861385079E8|         89738|
|1998|      3.668649685E7|         14390|
|1998|      1.662960392E7|          5787|
|1998|       8.52247009E7|         22818|
|1998|      4.393428823E8|         99080|
|1998|         6353589.59|          4317|
+----+-------------------+--------

In [33]:
df.groupBy('year').mean().show()

+----+---------+------------------+------------------------+-------------------+
|year|avg(Year)|      avg(Quarter)|avg(Total visitor spend)|avg(Total visitors)|
+----+---------+------------------+------------------------+-------------------+
|2003|   2003.0|               2.5|     5.135416301498114E7| 13634.035849056603|
|2007|   2007.0|               2.5|     5.298485946447189E7| 15404.785211267606|
|2018|   2018.0|               1.0|     7.026104768109676E7| 21725.135483870967|
|2015|   2015.0|               2.5|     5.966558845162526E7| 18085.746268656716|
|2006|   2006.0|2.5018050541516246|    5.0525060003616996E7| 15063.421985815603|
|2013|   2013.0|               2.5|     4.421295895952306E7| 16116.270868824531|
|1997|   1997.0|               4.0|    3.3225332315086212E7| 11587.715517241379|
|2014|   2014.0|               2.5|    4.5876300145355344E7| 16643.788429752065|
|2004|   2004.0|               2.5|     5.061080516626356E7| 14528.119133574008|
|1998|   1998.0|            

In [34]:
df.groupBy('year').sum().show()

+----+---------+------------+------------------------+-------------------+
|year|sum(Year)|sum(Quarter)|sum(Total visitor spend)|sum(Total visitors)|
+----+---------+------------+------------------------+-------------------+
|2003|  1666496|        2080|    2.721770639794000...|            7226039|
|2007|  1669824|        2080|    3.009540017582003...|            8749918|
|2018|   419744|         208|    1.089046239056999...|            3367396|
|2015|  1676480|        2080|    3.597834983633003E10|           10905705|
|2006|  1666986|        2079|    2.849613384203998...|            8495770|
|2013|  1674816|        2080|    2.595300690924003...|            9460251|
|1997|   415376|         832|    3.8541385485500007E9|            1344175|
|2014|  1675648|        2080|    2.775516158793998...|           10069492|
|2004|  1667328|        2080|    2.803838606211001...|            8048578|
|1998|  1662336|        2080|    1.575102648142000...|            5260311|
|2012|  1673984|        2

In [35]:
from pyspark.sql.functions import format_number, col

# Let's start off with this. Just grouping by year and presenting the mean.
group_year_df = df.groupBy('year').mean()
group_year_df.show()

# We can use format_number to reduce the total amount of decimals. 
# The number two represents the amount of decimals we want to be displayed.
group_job_df = group_job_df.select('year',
                                   format_number('avg(total visitor spend)',2),
                                   format_number('avg(total visitors)',2))
group_job_df.show()

+----+---------+------------------+------------------------+-------------------+
|year|avg(Year)|      avg(Quarter)|avg(Total visitor spend)|avg(Total visitors)|
+----+---------+------------------+------------------------+-------------------+
|2003|   2003.0|               2.5|     5.135416301498114E7| 13634.035849056603|
|2007|   2007.0|               2.5|     5.298485946447189E7| 15404.785211267606|
|2018|   2018.0|               1.0|     7.026104768109676E7| 21725.135483870967|
|2015|   2015.0|               2.5|     5.966558845162526E7| 18085.746268656716|
|2006|   2006.0|2.5018050541516246|    5.0525060003616996E7| 15063.421985815603|
|2013|   2013.0|               2.5|     4.421295895952306E7| 16116.270868824531|
|1997|   1997.0|               4.0|    3.3225332315086212E7| 11587.715517241379|
|2014|   2014.0|               2.5|    4.5876300145355344E7| 16643.788429752065|
|2004|   2004.0|               2.5|     5.061080516626356E7| 14528.119133574008|
|1998|   1998.0|            

NameError: name 'group_job_df' is not defined

In [36]:
from pyspark.sql.functions import format_number, col

# Let's start off with this. Just grouping by year and presenting the mean.
group_year_df = df.groupBy('year').mean()
group_year_df.show()

# We can use format_number to reduce the total amount of decimals. 
# The number two represents the amount of decimals we want to be displayed.
group_year_df = group_year_df.select('year',
                                   format_number('avg(total visitor spend)',2),
                                   format_number('avg(total visitors)',2))
group_year_df.show()

+----+---------+------------------+------------------------+-------------------+
|year|avg(Year)|      avg(Quarter)|avg(Total visitor spend)|avg(Total visitors)|
+----+---------+------------------+------------------------+-------------------+
|2003|   2003.0|               2.5|     5.135416301498114E7| 13634.035849056603|
|2007|   2007.0|               2.5|     5.298485946447189E7| 15404.785211267606|
|2018|   2018.0|               1.0|     7.026104768109676E7| 21725.135483870967|
|2015|   2015.0|               2.5|     5.966558845162526E7| 18085.746268656716|
|2006|   2006.0|2.5018050541516246|    5.0525060003616996E7| 15063.421985815603|
|2013|   2013.0|               2.5|     4.421295895952306E7| 16116.270868824531|
|1997|   1997.0|               4.0|    3.3225332315086212E7| 11587.715517241379|
|2014|   2014.0|               2.5|    4.5876300145355344E7| 16643.788429752065|
|2004|   2004.0|               2.5|     5.061080516626356E7| 14528.119133574008|
|1998|   1998.0|            

In [37]:
# But now the column names look quite unprofessional. We can assign an alias to rename each of them.
group_year_df = group_year_df.select(col('year').alias('Year'),
                                   col('format_number(avg(total visitor spend), 2)').alias('Average Spending'),
                                   col('format_number(avg(total visitors), 2)').alias('Average Visitors'))
group_year_df.show()

# Finally, let's sort the DataFrame by age.
group_year_df = group_year_df.orderBy('Year')

print('Average Spending and Visitors by Year')
group_year_df.show()

+----+----------------+----------------+
|Year|Average Spending|Average Visitors|
+----+----------------+----------------+
|2003|   51,354,163.01|       13,634.04|
|2007|   52,984,859.46|       15,404.79|
|2018|   70,261,047.68|       21,725.14|
|2015|   59,665,588.45|       18,085.75|
|2006|   50,525,060.00|       15,063.42|
|2013|   44,212,958.96|       16,116.27|
|1997|   33,225,332.32|       11,587.72|
|2014|   45,876,300.15|       16,643.79|
|2004|   50,610,805.17|       14,528.12|
|1998|   33,370,818.82|       11,144.73|
|2012|   47,985,850.44|       16,995.92|
|2009|   51,906,240.87|       15,241.98|
|2016|   65,215,792.57|       19,415.97|
|2001|   49,768,104.50|       13,240.63|
|2005|   49,971,741.26|       15,385.71|
|2000|   44,179,848.48|       12,080.99|
|2010|   48,563,938.15|       15,934.00|
|2011|   47,799,254.27|       16,327.36|
|2008|   52,318,622.95|       15,208.92|
|2017|   68,630,243.62|       21,632.17|
+----+----------------+----------------+
only showing top